In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, neighbors
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt

/Users/neelambabel/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:

df = pd.read_csv('/Users/neelambabel/ADS Folder/Assignment 2/voice-data.csv')

df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [5]:
df = df.rename(columns={'label': 'gender'})
df.columns.values

array(['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt',
       'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun',
       'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'gender'], dtype=object)

KNN Classifier

In [13]:
x = np.array(df.drop(['gender'], 1))
y = np.array(df['gender'])

#Dividing the data randomly into training and test set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model = neighbors.KNeighborsClassifier(3)
model.fit(x_train, y_train)

accuracy = model.score(x_test, y_test)
print('Accuracy='+str(accuracy))

Accuracy=0.725552050473


"""

The above was without any tuning , now we will drop some columns which do not make any sense.

We will drop col = median, mode, Q25, Q75, skew, kurt etc.

"""

In [14]:
df1=df[['meanfreq','sd','median','meanfun', 'IQR', 'gender']]
x = np.array(df1.drop(['gender'], 1))
y = np.array(df1['gender'])

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model = neighbors.KNeighborsClassifier(9)
model.fit(x_train, y_train)

accuracy2 = model.score(x_test, y_test)
print('Accuracy2='+str(accuracy2))


Accuracy2=0.98738170347


Improving the model using Feature Selection from Sklearn

In [8]:
from sklearn.feature_selection import SelectKBest, f_classif

def select_kbest_clf(data_frame, target, k=5):
    """
    Selecting K-Best features for classification
    :param data_frame: A pandas dataFrame with the training data
    :param target: target variable name in DataFrame
    :param k: desired number of features from the data
    :returns feature_scores: scores for each feature in the data as 
    pandas DataFrame
    """
    feat_selector = SelectKBest(f_classif, k=k)
    _ = feat_selector.fit(data_frame.drop(target, axis=1), data_frame[target])
    
    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = feat_selector.scores_
    feat_scores["P Value"] = feat_selector.pvalues_
    feat_scores["Support"] = feat_selector.get_support()
    feat_scores["Attribute"] = data_frame.drop(target, axis=1).columns
    
    return feat_scores

    
k=select_kbest_clf(df, 'gender', k=5)
k


,F Score,P Value,Support,Attribute
0,406.752821,3.368950e-85,False,meanfreq
1,945.461378,6.654751e-182,True,sd
2,277.588158,8.259210e-60,False,median
3,1121.569223,9.140835e-211,True,Q25
4,14.236082,1.642021e-04,False,Q75
5,1965.749999,0.000000e+00,True,IQR
6,4.252980,3.926293e-02,False,skew
7,24.255365,8.869557e-07,False,kurt
8,1003.308717,1.614016e-191,True,sp.ent
9,463.923194,3.877715e-96,False,sfm


In [12]:
df3=df[['meanfun','IQR','Q25','sp.ent','sd','sfm','meanfreq','gender']]
x = np.array(df3.drop(['gender'], 1))
y = np.array(df3['gender'])

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model = neighbors.KNeighborsClassifier(6)
model.fit(x_train, y_train)

accuracy3 = model.score(x_test, y_test)
print('Accuracy3='+str(accuracy3))

Accuracy3=0.979495268139


Decision Trees

In [17]:
def genderToInt(gender):
    if gender == 'male':
        return 0
    else:
        return 1
    
df.gender = df.gender.apply(genderToInt)
print(df.head())
print(df.tail())



   meanfreq        sd    median       Q25       Q75       IQR       skew  \
0  0.059781  0.064241  0.032027  0.015071  0.090193  0.075122  12.863462   
1  0.066009  0.067310  0.040229  0.019414  0.092666  0.073252  22.423285   
2  0.077316  0.083829  0.036718  0.008701  0.131908  0.123207  30.757155   
3  0.151228  0.072111  0.158011  0.096582  0.207955  0.111374   1.232831   
4  0.135120  0.079146  0.124656  0.078720  0.206045  0.127325   1.101174   

          kurt    sp.ent       sfm   ...    centroid   meanfun    minfun  \
0   274.402906  0.893369  0.491918   ...    0.059781  0.084279  0.015702   
1   634.613855  0.892193  0.513724   ...    0.066009  0.107937  0.015826   
2  1024.927705  0.846389  0.478905   ...    0.077316  0.098706  0.015656   
3     4.177296  0.963322  0.727232   ...    0.151228  0.088965  0.017798   
4     4.333713  0.971955  0.783568   ...    0.135120  0.106398  0.016931   

     maxfun   meandom    mindom    maxdom   dfrange   modindx  gender  
0  0.275862  0

In [18]:
x = np.array(df.drop(['gender'], 1))
y = np.array(df['gender'])

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier

#DecisionTreeClassifier

dt=DecisionTreeClassifier(max_depth=3, min_samples_leaf=int(0.5*len(x_train)))
boosted_dt=AdaBoostClassifier(dt,algorithm='SAMME', n_estimators=800, learning_rate=0.5)
boosted_dt.fit(x_train,y_train)
y_predicted=boosted_dt.predict(x_test)

print ("Area under ROC curve: %.4f"%(roc_auc_score(y_test, y_predicted)))

Area under ROC curve: 0.9495
